In [3]:
# import

from transformers import PreTrainedTokenizerFast
import os
from tqdm import tqdm
import json
import csv
import pandas as pd

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

In [31]:
# 주제별 텍스트 일상 대화 데이터

result_txt_dir = "주제별 텍스트 일상 대화 데이터.txt"

if os.path.exists(result_txt_dir):
    os.remove(result_txt_dir)

base_dir = "/workspace/020.주제별 텍스트 일상 대화 데이터/01.데이터"
train_dir = os.path.join(base_dir, "1.Training/원천데이터")
val_dir = os.path.join(base_dir, "2.Validation/원천데이터")

train_dir_list = os.listdir(train_dir)
val_dir_list = os.listdir(val_dir)

for dir in tqdm(train_dir_list, total=len(train_dir_list)):
    current_dir = os.path.join(train_dir, dir)
    
    txt_list = os.listdir(current_dir)
    
    for txt in txt_list:
        current_txt_dir = os.path.join(current_dir, txt)
        
        with open(current_txt_dir, 'r', encoding='utf-8') as f1, open(result_txt_dir, 'a', encoding='utf-8') as f2:
            for line in f1.readlines():
                if line[0].isdigit() and len(tokenizer.encode(line, add_special_tokens=False)) > 3:
                    f2.write(line[4::].strip() + '\n')
                    
for dir in tqdm(val_dir_list, total=len(val_dir_list)):
    current_dir = os.path.join(val_dir, dir)
    
    txt_list = os.listdir(current_dir)
    
    for txt in txt_list:
        current_txt_dir = os.path.join(current_dir, txt)
        
        with open(current_txt_dir, 'r', encoding='utf-8') as f1, open(result_txt_dir, 'a', encoding='utf-8') as f2:
            for line in f1.readlines():
                if line[0].isdigit() and len(tokenizer.encode(line, add_special_tokens=False)) > 3:
                    f2.write(line[4::].strip() + '\n')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:29<00:00,  5.99s/it]


In [33]:
# 소상공인 고객 주문 질의-응답 텍스트

result_txt_dir = "소상공인 고객 주문 질의-응답 텍스트.txt"

if os.path.exists(result_txt_dir):
    os.remove(result_txt_dir)

train_dir = "/workspace/소상공인 고객 주문 질의-응답 텍스트/Training/라벨링데이터_train"
val_dir = "/workspace/소상공인 고객 주문 질의-응답 텍스트/Validation/라벨링데이터_validation"

train_dir_list = os.listdir(train_dir)
val_dir_list = os.listdir(val_dir)

dataset = []

for dir in tqdm(train_dir_list, total=len(train_dir_list)):
    current_json_dir = os.path.join(train_dir, dir)
    data = pd.read_csv(current_json_dir, keep_default_na=False)

    for text in data['발화문']:
        if len(tokenizer.encode(text, add_special_tokens=False)) > 3:
            dataset.append(text)

for dir in val_dir_list:
    current_json_dir = os.path.join(val_dir, dir)
    data = pd.read_csv(current_json_dir, keep_default_na=False)

    for text in data['발화문']:
        if len(tokenizer.encode(text, add_special_tokens=False)) > 3:
            dataset.append(text)
            
with open(result_txt_dir, 'a', encoding='utf-8') as f:
    for d in dataset:
        f.write(d.strip() + '\n')

  7%|██████████▌                                                                                                                                        | 1/14 [00:08<01:56,  8.93s/it]/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [05:47<00:00, 24.86s/it]


In [34]:
# 한국어 SNS

result_txt_dir = "한국어 SNS.txt"

if os.path.exists(result_txt_dir):
    os.remove(result_txt_dir)

train_dir = "/workspace/한국어 SNS/Training/[라벨]한국어SNS_train"
val_dir = "/workspace/한국어 SNS/Validation/[라벨]한국어SNS_valid"

train_dir_list = os.listdir(train_dir)
val_dir_list = os.listdir(val_dir)

sns_dataset = []

for dir in train_dir_list:
    current_json_dir = os.path.join(train_dir, dir)

    with open(current_json_dir, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    for data in tqdm(json_data['data'], total=len(json_data['data'])):
        for body in data['body']:
            if len(tokenizer.encode(body['utterance'], add_special_tokens=False)) > 3:
                sns_dataset.append(body['utterance'])

for dir in val_dir_list:
    current_json_dir = os.path.join(val_dir, dir)

    with open(current_json_dir, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    for data in tqdm(json_data['data'], total=len(json_data['data'])):
        for body in data['body']:
            if len(tokenizer.encode(body['utterance'], add_special_tokens=False)) > 3:
                sns_dataset.append(body['utterance'])
    
with open(result_txt_dir, 'a', encoding='utf-8') as f:
    for sns_data in sns_dataset:
        f.write(sns_data.strip() + '\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12423/12423 [00:10<00:00, 1232.51it/s]


In [4]:
import re
from itertools import groupby

def repeat(data):
    return [[k, len(list(g))] for k, g in groupby(data)]

total_dataset = []

with open('주제별 텍스트 일상 대화 데이터.txt', 'r', encoding='utf-8') as f1:
    lines = f1.readlines()
    for line in tqdm(lines, total=len(lines)):
        w_list = repeat(line.strip())
        for i in range(len(w_list)):
            if w_list[i][1] > 2:
                 w_list[i][1] = 2
                    
        s = ''
        for w in w_list:
            s += w[0] * w[1]
            
        if s.find('#@') == -1 and len(tokenizer.encode(s, add_special_tokens=False)) > 3:
            total_dataset.append(s)

with open('소상공인 고객 주문 질의-응답 텍스트.txt', 'r', encoding='utf-8') as f1:
    lines = f1.readlines()
    for line in tqdm(lines, total=len(lines)):
        w_list = repeat(line.strip())
        for i in range(len(w_list)):
            if w_list[i][1] > 2:
                 w_list[i][1] = 2
                    
        s = ''
        for w in w_list:
            s += w[0] * w[1]
            
        if s.find('#@') == -1 and len(tokenizer.encode(s, add_special_tokens=False)) > 3:
            total_dataset.append(s)
            
with open('한국어 SNS.txt', 'r', encoding='utf-8') as f1:
    lines = f1.readlines()
    for line in tqdm(lines, total=len(lines)):
        w_list = repeat(line.strip())
        for i in range(len(w_list)):
            if w_list[i][1] > 2:
                 w_list[i][1] = 2
                    
        s = ''
        for w in w_list:
            s += w[0] * w[1]
            
        if s.find('#@') == -1 and len(tokenizer.encode(s, add_special_tokens=False)) > 3:
            total_dataset.append(s)

with open('dialogue_corpus_bak.txt', 'r', encoding='utf-8') as f1:
    lines = f1.readlines()
    for line in tqdm(lines, total=len(lines)):
        w_list = repeat(line.strip())
        for i in range(len(w_list)):
            if w_list[i][1] > 2:
                 w_list[i][1] = 2
                    
        s = ''
        for w in w_list:
            s += w[0] * w[1]
            
        if s.find('#@') == -1 and len(tokenizer.encode(s, add_special_tokens=False)) > 3:
            total_dataset.append(s)
            
print(len(total_dataset))

with open('dialogue_corpus_a.txt', 'w', encoding='utf-8') as f:
    for data in total_dataset:
        f.write(data.strip()+'\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93146044/93146044 [3:31:34<00:00, 7337.68it/s]


118590872


In [5]:
dataset_final = []

with open('dialogue_corpus_a.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        dataset_final.append(line.strip())
        
with open('dataset/emotion_and_minwon.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        dataset_final.append(line.strip())

In [6]:
len(dataset_final)

119601330

In [8]:
with open('dataset/chitchat_corpus.txt', 'w', encoding='utf-8') as f:
    for data_final in tqdm(dataset_final, total=len(dataset_final)):
        f.write(data_final.strip() + '\n')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119601330/119601330 [01:51<00:00, 1075501.32it/s]
